In [1]:
import os
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
import pytorch_lightning as pl
trainer = pl.Trainer(devices=1, accelerator='gpu', max_epochs=180)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [2]:
# --- Config Information ---#
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel.yaml import YAML
config_path = './configs/config.yaml'

if not os.path.exists(config_path):
    # Grab the config we'll use in this example
    BRANCH = 'main'
    !mkdir configs
    !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', "o'", "g'", 'sh', 'ng', 'ch'], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', "o'", "g'", 'sh', 'ng', 'ch'], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', "o'", "g'", 'sh', 'ng', 'ch'], 

In [3]:
train_json_path = '/media/real/data/uzbekvoice/uzbekvoice-dataset/train.json'
test_json_path = '/media/real/data/uzbekvoice/uzbekvoice-dataset/test.json'

In [4]:
from omegaconf import DictConfig
params['model']['train_ds']['manifest_filepath'] = train_json_path
params['model']['validation_ds']['manifest_filepath'] = test_json_path
first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)

[NeMo I 2023-08-23 17:03:42 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-08-23 17:03:42 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-08-23 17:03:43 collections:196] Dataset loaded with 101117 files totalling 113.62 hours
[NeMo I 2023-08-23 17:03:43 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-08-23 17:03:43 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-08-23 17:03:43 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-08-23 17:03:43 collections:196] Dataset loaded with 25263 files totalling 28.34 hours
[NeMo I 2023-08-23 17:03:43 collections:197] 0 files were filtered total

In [22]:
# Start training!!!
trainer.fit(first_asr_model)

You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/real/PycharmProjects/NeMo/tutorials/asr/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-08-23 14:13:26 modelPT:721] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2023-08-23 14:13:26 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7fe9a5aa9ac0>" 
    will be used during training (effective maximum steps = 252800) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 252800
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 1.2 M 
2 | decoder           | ConvASRDecoder                    | 31.8 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.844     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=80` reached.


In [23]:
print(first_asr_model.transcribe(paths2audio_files=['/media/real/data/uzbekvoice/clips/0a0d61eb-90fc-4475-9f37-d3ea8791326a/8748bf7f5910ac05084fe52fc16f1cbf370d8b5c2d0fc7b46140a5efbdb8544b.wav'],
                                 batch_size=1))

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['tpurbodalarning raksariyati yashinhaqavaqttattalateshlayotgan bolgan ']


In [24]:
# Bigger batch-size = bigger throughput
params['model']['validation_ds']['batch_size'] = 16

# Setup the test data loader and make sure the model is on GPU
first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
first_asr_model.cuda()
first_asr_model.eval()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2023-08-23 16:59:43 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-08-23 16:59:43 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-08-23 16:59:44 collections:196] Dataset loaded with 25263 files totalling 28.34 hours
[NeMo I 2023-08-23 16:59:44 collections:197] 0 files were filtered totalling 0.00 hours
WER = 0.4756061296909383


In [ ]:
# Save the model
!mkdir -p ./saved_models
first_asr_model.save_to("./saved_models/")
!ls ./saved_models


In [ ]:
#load the model 

first_asr_model = nemo_asr.models.EncDecCTCModel.restore_from("./saved_models/JasperEncoderDecoder-STEP-243800.pt")




In [5]:
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")

[NeMo I 2023-08-23 17:04:07 cloud:58] Found existing object /home/real/.cache/torch/NeMo/NeMo_1.20.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.
[NeMo I 2023-08-23 17:04:07 cloud:64] Re-using file from: /home/real/.cache/torch/NeMo/NeMo_1.20.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo
[NeMo I 2023-08-23 17:04:07 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-08-23 17:04:08 features:289] PADDING: 16
[NeMo I 2023-08-23 17:04:08 save_restore_connector:249] Model EncDecCTCModel was successfully restored from /home/real/.cache/torch/NeMo/NeMo_1.20.0/QuartzNet15x5Base-En/2b066be39e9294d7100fb176ec817722/QuartzNet15x5Base-En.nemo.


In [6]:
# Check what kind of vocabulary/alphabet the model has right now
print(quartznet.decoder.vocabulary)


[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"]


In [7]:
quartznet.change_vocabulary(
    new_vocabulary=[
        ' ', 'a', 'b','d', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
        'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', "o'", "g'", "sh", "ng", "ch"
    ]
)

[NeMo I 2023-08-23 17:04:54 ctc_models:309] Changed decoder to output to [' ', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', "o'", "g'", 'sh', 'ng', 'ch'] vocabulary.


In [8]:
import copy
new_opt = copy.deepcopy(params['model']['optim'])
new_opt['lr'] = 0.001
first_asr_model.setup_optimization(optim_config=DictConfig(new_opt))
# And then you can invoke trainer.fit(first_asr_model)

[NeMo I 2023-08-23 17:04:57 modelPT:721] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )
[NeMo I 2023-08-23 17:04:57 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f859802d6a0>" 
    will be used during training (effective maximum steps = 568800) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 568800
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.001
     lr: 0.001
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f859802d6a0>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'val_loss',
  'reduce_on_plateau': False})

In [9]:
# Use the smaller learning rate we set before
quartznet.setup_optimization(optim_config=DictConfig(new_opt))

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# And now we can create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(devices=1, accelerator='gpu', max_epochs=20)
trainer.fit(quartznet)

[NeMo W 2023-08-23 17:04:59 modelPT:610] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2023-08-23 17:04:59 modelPT:721] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2023-08-23 17:04:59 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2023-08-23 17:04:59 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-08-23 17:04:59 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-08-23 17:05:00 collections:196] Dataset loaded with 101117 files totalling 113.62 hours
[NeMo I 2023-08-23 17:05:00 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-08-23 17:05:00 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-08-23 17:05:00 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-08-23 17:05:00 collections:196] Dataset loaded with 25263 files totalling 28.34 hours
[NeMo I 2023-08-23 17:05:00 collections:197] 0 files were filtered total

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [17]:
# print(first_asr_model.transcribe(paths2audio_files=['/media/real/data/uzbekvoice/clips/0a0d61eb-90fc-4475-9f37-d3ea8791326a/8748bf7f5910ac05084fe52fc16f1cbf370d8b5c2d0fc7b46140a5efbdb8544b.wav'],

print(quartznet.transcribe(paths2audio_files=['/media/real/data/uzbekvoice/clips/0a0d61eb-90fc-4475-9f37-d3ea8791326a/8748bf7f5910ac05084fe52fc16f1cbf370d8b5c2d0fc7b46140a5efbdb8544b.wav'],))


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['aaada aaa aaa aaaa aaaaatai bolgan']


In [ ]:
params['model']['validation_ds']['batch_size'] = 16

# Setup the test data loader and make sure the model is on GPU
quartznet.setup_test_data(test_data_config=params['model']['validation_ds'])
quartznet.cuda()
quartznet.eval()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in quartznet.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = quartznet(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        quartznet._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        quartznet._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")